# Gestor de investimento
---

In [2]:
import pandas as pd
import numpy as np
import os
import xlrd

import plotly.graph_objs as go

# Config
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
# Função para pegar os dados de Papel (ações)
def get_stocks(df):
    stocks = []

    for index, row in df[~df['Unnamed: 2'].isnull()].iterrows():
        if row['Unnamed: 2'] == 'Papel' or row['Unnamed: 2'] == 'Ações':
            continue
        elif row['Unnamed: 2'] == 'Opções':
            break
        else:
            stocks.append(row[['Unnamed: 2', 'Unnamed: 10', 'Unnamed: 18', 
                                'Unnamed: 28', 'Unnamed: 35', 'Unnamed: 43', 
                                'Unnamed: 51', 'Unnamed: 61', 'Unnamed: 68', 
                                'Unnamed: 76', 'Unnamed: 83']].values)

    pd_stock = pd.DataFrame(columns=['Papel', 'Qtd Disponivel', 
                                    'Qtd Projetado', 'Qtd Dia', 
                                    'Qtd Garantia BOV', 'Qtd Garantia BMF', 
                                    'Qtd Estruturados', 'Liq Termo', 
                                    'Qtd Total', 'Cotacao', 'Financeiro'], data=stocks)

    return pd_stock


def get_picking(df):
    picking = []
    start = False

    for index, row in df[~df['Unnamed: 2'].isnull()].iterrows():
        if row['Unnamed: 2'] == 'Proventos de Ação':
                start = True
                continue
        elif row['Unnamed: 2'] == 'Papel':
            continue
        elif row['Unnamed: 2'] == 'Renda Fixa':
            break
        elif start == True:
            picking.append(row[['Unnamed: 2', 'Unnamed: 11', 
                            'Unnamed: 22', 'Unnamed: 60', 
                            'Unnamed: 77']].values)

    pd_picking = pd.DataFrame(columns=['Papel', 'Qtd Provisionada', 
                                    'Tipo', 'Data Pagamento', 'Valor'], data=picking)
  
    return pd_picking


def get_fi(df):
    x = []
    start = False

    for index, row in df[~df.isnull()].iterrows():
        if row['Unnamed: 2'] == 'Fundos de Investimentos':
                start = True
                continue
        elif row['Unnamed: 2'] == 'Nome Fundo':
            continue
        elif row['Unnamed: 2'] == 'Posição de Fundos Imobiliários':
            break
        elif (start == True) and ((type(row['Unnamed: 2']) == str)):
            if type(row['Unnamed: 2']) == str: 
                fi_name = np.array([row['Unnamed: 2']])
                data = df[df.index == index +1][['Unnamed: 13', 'Unnamed: 19', 
                                                    'Unnamed: 33', 'Unnamed: 40', 
                                                    'Unnamed: 48', 'Unnamed: 54', 
                                                    'Unnamed: 60', 'Unnamed: 70', 
                                                    'Unnamed: 81']]
                
                data.insert(0, 'Nome FI', row['Unnamed: 2'])
                # print(data.values[0])
                x.append(data.values[0])
                #display(data.head())
    
    pd_fi = pd.DataFrame(columns=['Nome', 'Data', 'Qtd Cotas', 
                                    'Valor Cota', 'Valor Bruto', 
                                    'IR', 'IOF', 'Valor Liquido', 
                                    'Aplicacao Pendente', 'Total Bruto'], data=x)
    return pd_fi


def get_fii(df):
    fii = []
    start = False

    for index, row in df[~df['Unnamed: 2'].isnull()].iterrows():
        if row['Unnamed: 2'] == 'Posição de Fundos Imobiliários':
                start = True
                continue
        elif row['Unnamed: 2'] == 'Nome':
            continue
        elif row['Unnamed: 2'] == 'Proventos de Fundo Imobiliário':
            break
        elif start == True:
            fii.append(row[['Unnamed: 2', 'Unnamed: 14', 
                            'Unnamed: 26', 'Unnamed: 38', 
                            'Unnamed: 45', 'Unnamed: 55', 
                            'Unnamed: 74']].values)

    df_result = pd.DataFrame(columns=['Papel', 'Qtd Disponivel', 
                                    'Qtd Projetada', 'Qtd Dia', 'Qtde Total', 
                                    'Ult Cotacao', 'Financeiro'], data=fii)
  
    return df_result


def get_picking_fii(df):
    p_fii = []
    start = False

    for index, row in df[~df['Unnamed: 2'].isnull()].iterrows():
        if row['Unnamed: 2'] == 'Proventos de Fundo Imobiliário':
                start = True
                continue
        elif row['Unnamed: 2'] == 'Papel':
            continue
        elif row['Unnamed: 2'] == 'Clubes de Investimentos':
            break
        elif start == True:
            p_fii.append(row[['Unnamed: 2', 'Unnamed: 11', 
                            'Unnamed: 22', 'Unnamed: 60', 
                            'Unnamed: 77']].values)

    df_result = pd.DataFrame(columns=['Papel', 'Tipo', 
                                    'Qtd Provisionada', 'Dt Pagamento', 
                                    'Valor Provisionado'], data=p_fii)
  
    return df_result


def get_extract():
    df = pd.read_csv('../datasets/extrato/Extrato2.csv', sep=';', encoding='iso-8859-1', decimal=',')
    return df

    

In [5]:
files = os.listdir('../datasets/posicao/')

stocks = pd.DataFrame()
pickings = pd.DataFrame()
fis = pd.DataFrame()
fiis = pd.DataFrame()
picking_fii = pd.DataFrame()

for file_name in files:
    if file_name == '.DS_Store':
        continue
    
    wb = xlrd.open_workbook('../datasets/posicao/' + file_name, logfile=open(os.devnull, 'w'))
    df = pd.read_excel(wb)
    
    #df = pd.read_excel('../datasets/' + file_name)
    date_position = df[df['Unnamed: 56'].str.contains('Data de referência', na=False)]['Unnamed: 56']
    
    # position date
    date_position = pd.to_datetime(date_position.str.replace('Data de referência: ', ''), format='%d/%m/%Y')
    
    #month = int(date_position.dt.month.values)
    #year = int(date_position.dt.year.values)
    #period = str(year) + '/' + str(month)
    period = date_position.values[0]

    
    # get stocks
    df_stocks = get_stocks(df)
    #display(df_stocks.head())
    df_stocks['period'] = period
    stocks = stocks.append(df_stocks, ignore_index=True)

    df_pickings = get_picking(df)
    df_pickings['period'] = period
    pickings = pickings.append(df_pickings, ignore_index=True)

    df_fi = get_fi(df)
    df_fi['period'] = period
    fis = fis.append(df_fi, ignore_index=True)

    df_fii = get_fii(df)
    df_fii['period'] = period
    fiis = fiis.append(df_fii, ignore_index=True)

    df_picking_fii = get_picking_fii(df)
    df_picking_fii['period'] = period

    picking_fii = picking_fii.append(df_picking_fii, ignore_index=True)

# display(stocks.sort_values('period', ascending=False))
# display(fi)

In [6]:
picking_fii.sort_values(by='period').head()

,Papel,Tipo,Qtd Provisionada,Dt Pagamento,Valor Provisionado,period
17,RNGO11,RENDIMENTO,49,14/02/2017,30.38,2017-01-31
16,RNGO11,RENDIMENTO,49,14/03/2017,29.40,2017-02-24
23,RNGO11,RENDIMENTO,49,17/04/2017,29.40,2017-03-31
21,RNGO11,RENDIMENTO,49,15/05/2017,24.99,2017-04-28
12,RNGO11,RENDIMENTO,49,14/06/2017,26.46,2017-05-31


In [56]:
fis['period'].sort_values().unique()

array(['2016-04-29T00:00:00.000000000', '2016-05-31T00:00:00.000000000',
       '2016-06-30T00:00:00.000000000', '2016-07-29T00:00:00.000000000',
       '2016-08-31T00:00:00.000000000', '2016-09-30T00:00:00.000000000',
       '2016-10-31T00:00:00.000000000', '2016-11-30T00:00:00.000000000',
       '2016-12-29T00:00:00.000000000', '2017-01-31T00:00:00.000000000',
       '2017-02-24T00:00:00.000000000', '2017-03-31T00:00:00.000000000',
       '2017-04-28T00:00:00.000000000', '2017-05-31T00:00:00.000000000',
       '2017-06-30T00:00:00.000000000', '2017-07-31T00:00:00.000000000',
       '2017-08-31T00:00:00.000000000', '2017-09-29T00:00:00.000000000',
       '2017-10-31T00:00:00.000000000', '2017-11-30T00:00:00.000000000',
       '2017-12-28T00:00:00.000000000', '2018-01-31T00:00:00.000000000',
       '2018-02-28T00:00:00.000000000', '2018-03-29T00:00:00.000000000',
       '2018-04-30T00:00:00.000000000', '2018-05-30T00:00:00.000000000',
       '2018-06-29T00:00:00.000000000', '2018-07-31

In [8]:
display(stocks.head())
display(pickings.head())
display(fis.head())
display(fiis.head())
display(picking_fii.head())

,Papel,Qtd Disponivel,Qtd Projetado,Qtd Dia,Qtd Garantia BOV,Qtd Garantia BMF,Qtd Estruturados,Liq Termo,Qtd Total,Cotacao,Financeiro,period
0,JHSF3,1600,0,0,0,0,0,0,1600,7.01,"11,216.00",2020-02-28
1,MGLU3,800,0,0,0,0,0,0,800,50.52,"40,416.00",2020-02-28
2,MGLU3,800,0,0,0,0,0,0,800,37.04,"29,632.00",2019-09-30
3,BPAC11,200,0,0,0,0,0,0,200,42.30,"8,460.00",2020-04-30
4,JHSF3,1600,0,0,0,0,0,0,1600,4.16,"6,656.00",2020-04-30


,Papel,Qtd Provisionada,Tipo,Data Pagamento,Valor,period
0,MGLU3,800,JUROS SOBRE CAPITAL PROPRIO,Indeterminado,28.63,2020-02-28
1,MGLU3,800,JUROS SOBRE CAPITAL PROPRIO,Indeterminado,58.88,2020-02-28
2,MGLU3,800,JUROS SOBRE CAPITAL PROPRIO,Indeterminado,28.63,2020-04-30
3,MGLU3,800,JUROS SOBRE CAPITAL PROPRIO,Indeterminado,58.88,2020-04-30
4,WEGE3,300,JUROS SOBRE CAPITAL PROPRIO,12/08/2020,9.07,2020-04-30


,Nome,Data,Qtd Cotas,Valor Cota,Valor Bruto,IR,IOF,Valor Liquido,Aplicacao Pendente,Total Bruto,period
0,XP Long Short FIC FIM,2020-02-28,"6,902.02",2.91,"20,105.88",38.48,0,"20,067.40",0,"20,105.88",2020-02-28
1,Equitas Selection FIC FIA,2020-02-28,"3,018.52",7.23,"21,812.68",286.90,0,"21,525.78",0,"21,812.68",2020-02-28
2,AZ Quest Multi FIC FIM,2020-02-28,"1,464.80",12.95,"18,967.40",93.33,0,"18,874.07",0,"18,967.40",2020-02-28
3,Mauá Macro FIC FIM,2020-02-28,37.17,787.62,"29,278.24",186.19,0,"29,092.05",0,"29,278.24",2020-02-28
4,Bahia AM Maraú FIC de FIM,2020-02-28,"33,810.43",2.56,"86,466.95",67.77,0,"86,399.18",0,"86,466.95",2020-02-28


,Papel,Qtd Disponivel,Qtd Projetada,Qtd Dia,Qtde Total,Ult Cotacao,Financeiro,period
0,IBFF11,50,0,0,50.00,100.10,"5,005.00",2020-02-28
1,BCFF11,8,0,0,8.00,98.17,785.36,2020-02-28
2,CNES11,11,0,0,11.00,83.50,918.50,2020-02-28
3,OULG11B,50,0,0,50.00,105.00,"5,250.00",2020-02-28
4,RNGO11,49,0,0,49.00,96.61,"4,733.89",2020-02-28


,Papel,Tipo,Qtd Provisionada,Dt Pagamento,Valor Provisionado,period
0,RNGO11,RENDIMENTO,49,13/03/2020,23.03,2020-02-28
1,RNGO11,RENDIMENTO,49,14/10/2019,24.01,2019-09-30
2,RNGO11,RENDIMENTO,49,14/08/2019,24.01,2019-07-31
3,RNGO11,RENDIMENTO,49,15/07/2019,26.95,2019-06-28
4,RNGO11,RENDIMENTO,49,15/05/2020,20.58,2020-04-30


# Analise de fundo imobiliario

In [9]:
fis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232 entries, 0 to 231
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Nome                232 non-null    object        
 1   Data                232 non-null    datetime64[ns]
 2   Qtd Cotas           232 non-null    float64       
 3   Valor Cota          232 non-null    float64       
 4   Valor Bruto         232 non-null    float64       
 5   IR                  232 non-null    float64       
 6   IOF                 232 non-null    object        
 7   Valor Liquido       232 non-null    float64       
 8   Aplicacao Pendente  232 non-null    object        
 9   Total Bruto         232 non-null    float64       
 10  period              232 non-null    datetime64[ns]
dtypes: datetime64[ns](2), float64(6), object(3)
memory usage: 20.1+ KB


In [10]:
fis.sort_values(['Nome', 'period']).head()

,Nome,Data,Qtd Cotas,Valor Cota,Valor Bruto,IR,IOF,Valor Liquido,Aplicacao Pendente,Total Bruto,period
133,AZ Quest Multi FIC FIM,2017-03-31,"1,516.42",9.95,"15,094.10",9.94,0,"15,034.29",0,"15,094.10",2017-03-31
122,AZ Quest Multi FIC FIM,2017-04-28,"1,516.42",10.00,"15,160.90",36.20,0,"15,124.70",0,"15,160.90",2017-04-28
82,AZ Quest Multi FIC FIM,2017-05-31,"1,516.42",9.73,"14,749.34",0.00,0,"14,749.34",0,"14,749.34",2017-05-31
86,AZ Quest Multi FIC FIM,2017-06-30,"1,516.42",9.80,"14,853.49",0.00,0,"14,853.49",0,"14,853.49",2017-06-30
108,AZ Quest Multi FIC FIM,2017-07-31,"1,516.42",10.15,"15,389.53",87.64,0,"15,301.89",0,"15,389.53",2017-07-31


In [20]:
fis[fis['Nome'] == 'AZ Quest Multi FIC FIM'].head()

,Nome,Data,Qtd Cotas,Valor Cota,Valor Bruto,IR,IOF,Valor Liquido,Aplicacao Pendente,Total Bruto,period
2,AZ Quest Multi FIC FIM,2020-02-28,"1,464.80",12.95,"18,967.40",93.33,0,"18,874.07",0,"18,967.40",2020-02-28
9,AZ Quest Multi FIC FIM,2019-09-30,"1,476.76",12.33,"18,214.14",107.71,0,"18,106.43",0,"18,214.14",2019-09-30
15,AZ Quest Multi FIC FIM,2019-07-31,"1,476.76",12.13,"17,914.04",62.70,0,"17,851.34",0,"17,914.04",2019-07-31
21,AZ Quest Multi FIC FIM,2019-06-28,"1,476.76",12.10,"17,872.39",56.45,0,"17,815.94",0,"17,872.39",2019-06-28
35,AZ Quest Multi FIC FIM,2017-11-30,"1,503.23",10.50,"15,784.72",46.16,0,"15,738.56",0,"15,784.72",2017-11-30


In [31]:
# Estou aqui!! Criar grafico comparando fundos por valor bruto

def plot_fis(df):

    for fi in df['Nome'].unique():
        fig = go.Figure()
        df_ = df[df['Nome'] == fi].sort_values('period')

        fig.add_trace(go.Scatter(x=df_['period'], y=df_['Qtd Cotas'] * df_['Valor Cota'], 
                                connectgaps=True,
                                mode='lines+markers',
                                name=fi, 
                                #text=round(df[mkt].astype(float)/1000000, 2).astype(str) + 'M', 
                                textposition="top center",
                                textfont={'color': 'rgb(100,100,100)', 'size':9}, 
                                # fill='tonexty', 
                                # df_offplatformByWeek['2020 10'].map('{:,.0f}'.format), 
                                marker=dict(size=3), 
                                opacity=.8))

        fig.update_layout(legend_orientation="h",
                        title=fi, 
                        height=600, 
                        width=1200, 
                        legend=dict(font=dict(size=9)), 
                        template='plotly_white'                
        )

        fig.show()

plot_fis(fis)

# Analise aportes

In [135]:
extract = get_extract()

In [136]:
extract.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Liq      340 non-null    object 
 1   Mov      340 non-null    object 
 2   Hist__o  340 non-null    object 
 3   Valor    340 non-null    float64
 4   Saldo    340 non-null    float64
dtypes: float64(2), object(3)
memory usage: 13.4+ KB


In [144]:
extract[extract['Valor'] > 22840]

,Liq,Mov,Hist__o,Valor,Saldo
61,12/09/2019,12/09/2019,TED - RECEBIMENTO DE TED - SPB,"47,000.00","47,547.52"
88,17/12/2018,17/12/2018,TED - RECEBIMENTO DE TED - SPB,"30,000.00","30,305.18"
141,05/10/2017,05/10/2017,TED - RECEBIMENTO DE TED - SPB,"73,000.00","73,259.50"
195,17/03/2017,17/03/2017,TED - RECEBIMENTO DE TED - SPB,"30,000.00","30,180.67"
251,31/03/2016,31/03/2016,RESGATE XP GlobCredit FICFIM,"31,029.08","31,185.91"


In [148]:
display(extract.groupby('Hist__o').agg({'Valor':'sum'}).reset_index().sort_values('Valor', ascending=False).head())
display(extract.groupby('Hist__o').agg({'Valor':'sum'}).reset_index().sort_values('Valor', ascending=False).tail(5)) # Aportes para fundos

,Hist__o,Valor
74,TED - RECEBIMENTO DE TED - SPB,"228,400.00"
52,RESGATE XP GlobCredit FICFIM,"31,029.08"
47,RESGATE MiraeMacroStrategy,"19,122.71"
53,RESGATE XP Macro FIM,"18,995.54"
49,RESGATE QuestMult FI CotasFI,"18,591.84"


,Hist__o,Valor
24,OPERAÍES EM BOLSA PR 12/09/2019 NOTA N¼ 16418593,"-27,630.26"
81,TED TER BCO 017 AGE 0001 CTA 1603 9 - TED APLI...,"-29,500.00"
78,TED TER BCO 017 AGE 0001 CTA 1310 2 - TED APLI...,"-30,000.00"
84,TED TER BCO 208 AGE 0001 CTA 183448 - TED APLI...,"-33,000.00"
85,TED TER BCO 237 AGE 2856 CTA 4563 2 - TED APLI...,"-73,000.00"


In [157]:
display(extract.groupby(['Hist__o', 'Mov']).agg({'Valor':'sum'}).reset_index().sort_values('Valor', ascending=False).tail(30).values)

array([['IRRF S/ RESGATE FUNDOS - COME COTAS Mau\x87 Macro FIC FIM',
        '01/06/2017', -109.38],
       ['IRRF S/ RESGATE FUNDOS - COME COTAS XP LONG SHORT',
        '01/06/2016', -132.97],
       ['IRRF S/ RESGATE FUNDOS - COME COTAS MiraeMacroStrategy',
        '05/12/2016', -180.48],
       ['OPERA\x82ÍES EM BOLSA LIQ. D+1 PR 24/05/2017 NOTA N¼ 8920878',
        '24/05/2017', -193.66],
       ['IRRF S/ RESGATE FUNDOS - COME COTAS MiraeMacroStrategy',
        '01/06/2017', -197.23],
       ['IRRF S/ RESGATE FUNDOS - COME COTAS XP LONG SHORT',
        '05/12/2016', -242.19],
       ['IRRF S/ RESGATE FUNDOS - COME COTAS Azul QuantitativoFIM',
        '01/06/2015', -251.79],
       ['DEPOSITO DE MARGEM DE GARANTIA', '19/06/2017', -578.89],
       ['TED BCO 341 AGE 0407 CTA 03400 8 - RETIRADA EM C/C',
        '03/12/2015', -3000.0],
       ['TED TER BCO 655 AGE 0001 CTA 1296001 5 - TED APLICA\x82ÌO FUNDOS Vot.FicFi CambialD',
        '10/09/2015', -3000.0],
       ['TED TER BCO 752 A